# Running AzureClusterlessHPC with Docker

This tutorial demonstrates how to run workloads with AzureClusterlessHPC using Azure Batch's container support. Instead of creating a deployment package for a VM image, we create a Docker container for the AzureClusterlessHPC runtime that is used by Azure Batch.

**Note:** This is different from running our local notebook from a Docker container. For instructions how to run your application from a Docker container, refer to the AzureClusterlessHPC documentation.


## Prepare Docker image

The first step is to create a container. The directory contains an example Dockerfile from which we can create an image as follows. From the command line, run:

```
Build local Docker image
docker build -t azureclusterlesshpc:v1.0 .
```

Once the image has been created, you can upload the image to your own image repository from where it will be downloaded from Azure Batch. First, log into your Docker account and then tag and upload the image. This step is optional for this tutorial,  as the example will use a public image that has already been created.

```
# Login
docker login

# Tag and push
docker tag azureclusterlesshpc:v1.0 username/azureclusterlesshpc:v1.0
docker push username/azureclusterlesshpc:v1.0
```

To specify a Docker image to use with AzureClusterlessHPC, add the environment variable `"_CONTAINER"` to the `parameters.json` file. For example, in the current example, we set `"_CONTAINER": "philippwitte/azureclusterlesshpc:v1.1"`. You must also supply the paths to the Julia and Python package directories. E.g., in our Dockerfile, we installed Julia into `/root`, so in our `parameters.json`, we set:

```
"_JULIA_DEPOT_PATH": "/root/.julia",
"_PYTHONPATH": "/usr/local/lib/python3.8/dist-packages"
```

## AzureClusterlessHPC example using Docker

As always, we start by setting the environment variables that point to our credentials and our `parameters.json` file:

In [1]:
# Set path to credentials
ENV["CREDENTIALS"] = joinpath(pwd(), "../..", "credentials.json")

# Set path to batch parameters (pool id, VM types, etc.)
ENV["PARAMETERS"] = joinpath(pwd(), "parameters.json")

# Load package
using AzureClusterlessHPC
batch_clear();

Next, we create the batch pool. Unlike before, we do not have to supply a start-up script or image ID for the VM, as we're running the AzureClusterlessHPC runtime from the Docker image that we created earlier. To speed-up the execution of tasks, Docker images are installed on VMs when they join the pool, not when a job or task is created.

In [2]:
# Create pool of VMs with pre-installed Docker images.
create_pool();

Created pool 1 of 1 in canadacentral with 2 nodes.


We define a simple hello world function to execute via Azure Batch:

In [3]:
# Hello world example
@batchdef function hello_world(name)
    print("Hello $name\n")
    return "Goodbye"
end;

We execute the function in parallel on two workers, using the `@batchexex` macro and `pmap`:

In [4]:
# Execute function via Azure Batch
bctrl = @batchexec pmap(name -> hello_world(name), ["Jane", "Bob"]);

  3.862028 seconds (3.52 M allocations: 210.372 MiB, 2.22% gc time, 24.82% compilation time)


We then fetch the function output from the two workers and print it:

In [5]:
output = fetch(bctrl)
print(output);

Monitoring tasks for 'Completed' state, timeout in 60 minutes ...Creating job [BatchJob_bv3M5W4k_1]...
Uploading file /home/pwitte/.julia/dev/AzureClusterlessHPC/src/runtime/application-cmd to blob container [azureclusterlesstemp]...
Uploading file /home/pwitte/.julia/dev/AzureClusterlessHPC/src/runtime/batch_runtime.jl to blob container [azureclusterlesstemp]...
Uploading file task_1.dat to container [azureclusterlesstemp]...
Uploading file task_2.dat to container [azureclusterlesstemp]...
..................................................................................................................
Fetch output from task task_2................................
Fetch output from task task_1
Any["Goodbye", "Goodbye"]

Finally, we clean up all consumed Azure resources:

In [6]:
destroy!(bctrl);

## Copyright

Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License (MIT). See LICENSE in the repo root for license information.